# Nematic Order Statistics

## Run many simulations in parallel

In [1]:
import lammps2d as lmp
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as anm
import matplotlib.collections as clt
import pandas as pd
import string as st
import scipy.spatial as spp
import scipy.optimize as spo
import sympy as smp
import copy as cp
import pickle
import os
import sys
import datetime

from IPython.display import HTML

smp.init_printing()
%matplotlib inline
%reload_ext autoreload
%autoreload 2

The previous notebook results in a very noisy order parameter, where no clear trend of $S$ vs $B$ can be extracted. It's true that the final configuration depends on the intitial conditions which are all the same for our system, but there is also a stochastic contribution. This means that several realizations of the same test will result in varying order paramters, and we then need some statistics. 

In this notebook I make several experiments of the same parameters as before, which are the parameters that Dobnikar predicts as dimer forming. Since the experimetns take a while, I run them in parallel in six of the eight threads of a locally configured cluster. 

Configuring a local cluster is very easy with ipycluster. ipycluster can be installed using conda, and then we can run in DOS:

    !ipcluster start -n 6

which starts six parallel processes. 

The previous run of this notebook took conspicuosly little time. I have now realized that the reason this happened is that the packing was too low. Low packing simulations are faster because there are less particles within the cutoff which makes for less calculations. I must run this again. 

If the simulations take about 25 minutes, and I run 10 experiments with 20 fields, in six parallel threads, the total time will be around 13 hours.

In [2]:
25*10*20/6/60

13.88888888888889

This notebook was run in the lab computer. From the report of the finished simulations, it appears as if the whole process took about three hours. This is strange because it means that each simulation takes about 5 minute. In fact, these simulations take about 5 minutes, which is strange beacuse they appear to have the same conditions as the simulations of nb 19-StaticFieldNematicPhase, and those take about 25 minutes.

In [2]:
from IPython.display import clear_output
import ipyparallel as ipp
c = ipp.Client()
dview = c[:]

In [3]:
c[:].apply_sync(lambda : "Hello, World")

['Hello, World',
 'Hello, World',
 'Hello, World',
 'Hello, World',
 'Hello, World',
 'Hello, World']

In [4]:
%px import os; os.chdir('C://Users/aortiza/Dropbox/Colloids/SimulationsOfDimerSystem')

In [5]:
height = [4.046]
packing = [0.437]
field = np.linspace(1,20,20)
experiments = np.arange(0,10)

H,P,F,E = np.meshgrid(height,packing,field,experiments)

height = H.flatten()
packing = P.flatten()
field = F.flatten()
experiment = E.flatten()

In [6]:
%%px
import lammps2d as lmp
import numpy as np
import pickle
import datetime
import time

height = [4.046]
packing = [0.0437]
field = np.linspace(1,20,20)
experiments = np.arange(0,10)

H,P,F,E = np.meshgrid(height,packing,field,experiments)

height = H.flatten()
packing = P.flatten()
field = F.flatten()
experiment = E.flatten()

In [7]:
%%px 
def run_sim(i):
    angle = 0
    radius = 1.4
    region, initial_positions = lmp.initial_setup(n_of_particles = 150, packing=packing[i], height = height[i])

    sim = lmp.sim(initial_positions,
                  magnitude = field[i],
                  file_name = "field_%u"%field[i],
                  dir_name = "C:/Users/aortiza/Desktop/NematicPhase/exp_%u/"%experiment[i],
                  radius = radius,
                  diffusion = 0.073,
                  frequency=0,
                  susceptibility = 0.4,
                  angle=angle,
                  framerate=1e2,
                  timestep=0.5e-4,
                  total_time = 120,
                  temperature = 120,
                  stamp_time = False,
                  space = {'region':region,'boundary':['p','p','f'],'walls':[False,False,True]})
    
    """ Has the simulation been run to completion?"""
    filename = os.path.join(sim.sim_parameters.dir_name,sim.sim_parameters.file_name)
    
    if os.path.isfile(filename+".p") and os.path.isfile(filename+".lammpstrj"):
        
        os.path.isfile(filename+".lammpstrj")
        lz_rd = lmp.trj_lazyread(filename+".lammpstrj")
        last_frame_recorded = max(list(lz_rd.T.keys()))

        sim_p = pickle.load(open(filename+".p", "rb" ))
        last_frame_expected = sim_p.run_parameters.total_time/sim_p.run_parameters.timestep
        
        print(filename,end="")
        if last_frame_expected == last_frame_recorded:
            print(" was done")
            return
              
    print(filename,end="") 
    sim.generate_scripts()
    sim.run()
    print(" is done at " + str(datetime.datetime.now()))

In [ ]:
i = range(len(height))
amr = dview.map_async(lambda x: run_sim(x), i)
while not amr.wait(1):
    # clear output because we are printing the whole thing each time
    # rather than keeping track of what we've consumed
    clear_output(wait=True)
    for stdout in amr.stdout:
        sys.stdout.write(stdout)

C:/Users/aortiza/Desktop/NematicPhase/exp_0/field_1C:/Users/aortiza/Desktop/NematicPhase/exp_4/field_4C:/Users/aortiza/Desktop/NematicPhase/exp_8/field_7C:/Users/aortiza/Desktop/NematicPhase/exp_1/field_11C:/Users/aortiza/Desktop/NematicPhase/exp_4/field_14C:/Users/aortiza/Desktop/NematicPhase/exp_7/field_17

## Batch 2
Since they don't take as long as I expected, I will run a second batch to complete 20 experiments per point. 

In [8]:
height = [4.046]
packing = [0.0437]
field = np.linspace(1,20,20)
experiments = np.arange(10,20)

H,P,F,E = np.meshgrid(height,packing,field,experiments)

height = H.flatten()
packing = P.flatten()
field = F.flatten()
experiment = E.flatten()

In [9]:
%%px

height = [4.046]
packing = [0.0437]
field = np.linspace(1,20,20)
experiments = np.arange(10,20)

H,P,F,E = np.meshgrid(height,packing,field,experiments)

height = H.flatten()
packing = P.flatten()
field = F.flatten()
experiment = E.flatten()

In [ ]:
i = range(len(height))
amr = dview.map_async(lambda x: run_sim(x), i)
while not amr.wait(1):
    # clear output because we are printing the whole thing each time
    # rather than keeping track of what we've consumed
    clear_output(wait=True)
    for stdout in amr.stdout:
        sys.stdout.write(stdout)

C:/Users/aortiza/Desktop/NematicPhase/exp_10/field_1 is done at 2018-03-14 11:23:48.888000
C:/Users/aortiza/Desktop/NematicPhase/exp_11/field_1 is done at 2018-03-14 11:28:50.734000
C:/Users/aortiza/Desktop/NematicPhase/exp_12/field_1 is done at 2018-03-14 11:36:05.081000
C:/Users/aortiza/Desktop/NematicPhase/exp_13/field_1 is done at 2018-03-14 11:43:12.222000
C:/Users/aortiza/Desktop/NematicPhase/exp_14/field_1 is done at 2018-03-14 11:48:22.506000
C:/Users/aortiza/Desktop/NematicPhase/exp_15/field_1 is done at 2018-03-14 11:53:29.176000
C:/Users/aortiza/Desktop/NematicPhase/exp_16/field_1 is done at 2018-03-14 11:58:31.182000
C:/Users/aortiza/Desktop/NematicPhase/exp_17/field_1 is done at 2018-03-14 12:03:34.568000
C:/Users/aortiza/Desktop/NematicPhase/exp_18/field_1 is done at 2018-03-14 12:08:39.623000
C:/Users/aortiza/Desktop/NematicPhase/exp_19/field_1 is done at 2018-03-14 12:13:42.807000
C:/Users/aortiza/Desktop/NematicPhase/exp_10/field_2 is done at 2018-03-14 12:18:47.320000

# Why is it so fast in the lab computer and so slow in the office?

To test this, I will run a single experiment in both computers.

I have figured out that the problem with these simulations was that the packing was extremely low.

In [4]:
height = [4.046]
packing = [0.0437]
field = np.linspace(1,20,20)
experiments = np.arange(0,10)

H,P,F,E = np.meshgrid(height,packing,field,experiments)

height = H.flatten()
packing = P.flatten()
field = F.flatten()
experiment = E.flatten()

def run_sim(i):
    angle = 0
    radius = 1.4
    region, initial_positions = lmp.initial_setup(n_of_particles = 150, packing=packing[i], height = height[i])

    sim = lmp.sim(initial_positions,
                  magnitude = field[i],
                  file_name = "field_%u"%field[i],
                  dir_name = "C:/Users/aortiza/Desktop/exp_%u/"%experiment[i],
                  radius = radius,
                  diffusion = 0.073,
                  frequency=0,
                  susceptibility = 0.4,
                  angle=angle,
                  framerate=1e2,
                  timestep=0.5e-4,
                  total_time = 120,
                  temperature = 120,
                  stamp_time = False,
                  space = {'region':region,'boundary':['p','p','f'],'walls':[False,False,True]})
    
    """ Has the simulation been run to completion?"""
    filename = os.path.join(sim.sim_parameters.dir_name,sim.sim_parameters.file_name)
    
    if os.path.isfile(filename+".p") and os.path.isfile(filename+".lammpstrj"):
        
        os.path.isfile(filename+".lammpstrj")
        lz_rd = lmp.trj_lazyread(filename+".lammpstrj")
        last_frame_recorded = max(list(lz_rd.T.keys()))

        sim_p = pickle.load(open(filename+".p", "rb" ))
        last_frame_expected = sim_p.run_parameters.total_time/sim_p.run_parameters.timestep
        
        print(filename,end="")
        if last_frame_expected == last_frame_recorded:
            print(" was done")
            return
              
    print(filename,end="") 
    sim.generate_scripts()
    sim.run()
    print(" is done at " + str(datetime.datetime.now()))

This cell was run in the office computer.

In [11]:
%%time
run_sim(10)

C:/Users/aortiza/Desktop/exp_0/field_2 is done at 2018-03-14 11:02:02.003464
Wall time: 4min 49s


This cell was run in the Lab's computer

In [5]:
%%time
run_sim(10)

C:/Users/aortiza/Desktop/exp_0/field_2 is done at 2018-03-14 11:08:40.812000
Wall time: 4min 11s
